In [1]:
# Run a mock of fractal-demos/examples/01, with an additional
# illumination correction task which does *not* overwrite input
# images.

In [2]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST


def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"[image_data_exist_on_disk] {image.path} contains data")
        else:
            print(f"[image_data_exist_on_disk] {image.path} does *not* contain data")
            all_images_have_data = False
        
    return all_images_have_data

In [3]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmpmp0nvhzy/zarr_dir'


In [4]:
# Run create-ome-zarr (starting from empty Dataset)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr"],
            args=dict(image_dir="/tmp/input_images", zarr_dir=zarr_dir),
        )
    ],
    dataset=Dataset(),
)

# Images were added, but data do not exist on-disk yet
assert not image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmpmp0nvhzy/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 does *not* contain data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 does *not* contain data
/tmp/ipykernel_90334/771841664.py:16 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A_01',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0',
                attributes={
           

In [5]:
# Run yokogawa-to-zarr
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["yokogawa_to_zarr"],
            args=dict(),
        )
    ],
    dataset=dataset,
)

# Image data now exist on disk as well
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_90334/3959548577.py:16 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr',
            'yokogawa_to_zarr',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A_01',
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
          

In [6]:
# Run illumination correction, with overwrite_input=True
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction"],
            args=dict(overwrite_input=False),
        )
    ],
    dataset=dataset,
)

# Image data still exist (and they actually were modified)
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[illumination_correction] START
[illumination_correction] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0'
[illumination_correction] overwrite_input=True
[illumination_correction] subsets=None
[illumination_correction] END
[illumination_correction] START
[illumination_correction] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction] overwrite_input=True
[illumination_correction] subsets=None
[illumination_correction] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 contains data
/tmp/ipykernel_90334/3527970434.py:16 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr',
            'yokogawa_to_zarr',
            'illumination_correction',
        ],
        images=[
            SingleImage(
                path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
         

In [7]:
# Run new-ome-zarr, as an init task for MIP
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["new_ome_zarr"],
            args=dict(suffix="mip", project_to_2D=True),
        )
    ],
    dataset=dataset,
)

# 3D image data exist on disk
images_3D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 3
    ]
assert image_data_exist_on_disk(images_3D)

# 2D image data do not exist on disk
images_2D = [
        image
        for image in dataset.images
        if image.attributes["data_dimensionality"] == 2
    ]
assert not image_data_exist_on_disk(images_2D)

# Print dataset metadata
debug(dataset);

[new_ome_zarr] START
[new_ome_zarr] paths=['/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0', '/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0']
[new_ome_zarr] Identified old_plate='my_plate.zarr'
[new_ome_zarr] new_plate='my_plate_mip.zarr'
[new_ome_zarr] new_zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr'
[new_ome_zarr] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/01/0 does *not* contain data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/02/0 does *not* contain data
/tmp/ipykernel_90334/3692689075.py:29 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr',
            'yokogawa_to_zarr',
            'illumination_correction',
            'new_ome_zarr',
        ],
        images=[
            SingleImage(

In [8]:
# Run MIP task
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["maximum_intensity_projection"],
            args=dict(),
        )
    ],
    dataset=dataset,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[maximum_intensity_projection] START
[maximum_intensity_projection] old_zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0'
[maximum_intensity_projection] new_zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/01/0'
[maximum_intensity_projection] END
[maximum_intensity_projection] START
[maximum_intensity_projection] old_zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0'
[maximum_intensity_projection] new_zarr_path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/02/0'
[maximum_intensity_projection] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_90334/1172269738.py:16 <module>
    dataset: Dataset(
        id=None,
        history=[
        

In [9]:
# Run cellpose on current images
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["cellpose_segmentation"],
            args=dict(),
        )
    ],
    dataset=dataset,
)

# All image data (3D and 2D) now exists on disk
assert image_data_exist_on_disk(dataset.images)

# Print dataset metadata
debug(dataset);

[cellpose_segmentation] START
[cellpose_segmentation] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/01/0'
[cellpose_segmentation] out={}
[cellpose_segmentation] END
[cellpose_segmentation] START
[cellpose_segmentation] path='/tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/02/0'
[cellpose_segmentation] out={}
[cellpose_segmentation] END
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate.zarr/A/02/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/01/0 contains data
[image_data_exist_on_disk] /tmp/tmpmp0nvhzy/zarr_dir/my_plate_mip.zarr/A/02/0 contains data
/tmp/ipykernel_90334/1453350519.py:16 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr',
            'yokogawa_to_zarr',
            'illumination_correction',
            'new_ome_zarr',
            'maximum_intensity_projection',
            'cellpos